In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import messagebox
import pandas as pd
from ttkthemes import ThemedStyle

def process_excel_file(file_path):
    try:
        # Read Excel file into a DataFrame
        raw = pd.read_excel(file_path)

        # Drop rows where all values are NaN
        raw = raw.dropna(how='all')

        # Fill NaN values with an empty string
        raw = raw.fillna('')

        # Shift values in the specified column up by one cell
        raw['Taxable Amount'] = raw['Taxable Amount'].shift(-1)

        # Drop the last row if it contains NaN values
        raw = raw.iloc[:-1]

        # Create an empty DataFrame with specified columns
        columns = ['Mobile Number', 'Country Code', 'Name', 'Bill Number', 'Bill Amount',
                   'Date', 'Time', 'Dob', 'Email', 'Channel', 'Server', 'Table Counter']
        df = pd.DataFrame(columns=columns)

        # Update "Bill Number" column in 'df' with unique values from "Voucher" column in 'raw'
        df['Bill Number'] = raw['Voucher'].unique()

        # Group by "Bill Number" in raw_df and calculate the sum of "Bill Amount"
        bill_amount_sum = raw.groupby('Voucher')['Taxable Amount'].sum()

        # Map the sum values to the corresponding "Bill Number" in updated_dummy_df
        df['Bill Amount'] = df['Bill Number'].map(bill_amount_sum)

        # Map "Mobile Number" from raw_df to updated_dummy_df based on "Bill Number"
        mobile_number_mapping = raw.groupby('Voucher')['Customer Phone No'].first()
        df['Mobile Number'] = df['Bill Number'].map(mobile_number_mapping)

        # Map "Date" from raw_df to updated_dummy_df based on "Bill Number"
        date_mapping = raw.groupby('Voucher')['Date'].first()
        df['Date'] = df['Bill Number'].map(date_mapping)

        # Convert "Date" column to "YYYY-MM-DD" format
        df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

        # Map "Name" from raw_df to updated_dummy_df based on "Bill Number"
        name_mapping = raw.groupby('Voucher')['Customer Name'].first()
        df['Name'] = df['Bill Number'].map(name_mapping)
        
        # Map "Name" from raw_df to updated_dummy_df based on "Bill Number"
        name_mapping = raw.groupby('Voucher')['Beautician Name'].first()
        df['Server'] = df['Bill Number'].map(name_mapping)

        # Fill "Country Code" column with "91" only when "Mobile Number" is not blank
        df.loc[df['Mobile Number'].notnull(), 'Country Code'] = '91'

        # Fill "Time" column with "11:00:00"
        df['Time'] = '11:00:00'

        # Replace empty strings in "Bill Number" with NaN
        df['Bill Number'] = df['Bill Number'].replace('', pd.NA)

        # Remove rows where "Bill Number" is NaN
        df = df.dropna(subset=['Bill Number'])

        # Save the modified DataFrame to a CSV file
        save_path = filedialog.asksaveasfilename(
            defaultextension=".csv",
            filetypes=[("CSV files", "*.csv")],
            initialdir="/",  # Set your desired initial directory
            initialfile="output.csv",  # Set your desired initial file name
            title="Save As",
            master=root,  # Set the master to root for positioning
        )

        if save_path:
            df.to_csv(save_path, index=False)
            messagebox.showinfo("Success", f"Processed data saved to {save_path}")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

def browse_file(entry_widget):
    file_path = filedialog.askopenfilename(filetypes=[("Excel files", ".xlsx;.xls")])
    entry_widget.delete(0, tk.END)
    entry_widget.insert(0, file_path)

# GUI setup
root = tk.Tk()
root.title("Excel File Processor")

# Apply a Mac-like theme
style = ThemedStyle(root)
try:
    style.set_theme("aqua")
except tk.TclError:
    pass  # Ignore the error if the 'aqua' theme is not available

# Set the initial position and size of the root window (left upper side and larger size)
root.geometry("800x600+0+0")  # Width x Height + X position + Y position

# Raw Excel file entry
raw_label = ttk.Label(root, text="Excel File:")
raw_label.grid(row=0, column=0, padx=10, pady=10)
raw_entry = ttk.Entry(root, width=50)
raw_entry.grid(row=0, column=1, padx=10, pady=10)
raw_button = ttk.Button(root, text="Browse", command=lambda: browse_file(raw_entry))
raw_button.grid(row=0, column=2, padx=10, pady=10)

# Process button
process_button = ttk.Button(root, text="Process Excel", command=lambda: process_excel_file(raw_entry.get()))
process_button.grid(row=1, column=1, pady=20)

root.mainloop()